# Preprocessing and Quality Control of NMR Data with the nPYc-Toolbox

This tutorial demonstrates how to use the NMR data processing modules of the nPYc-Toolbox, to import and perform some basic preprocessing and quality control of NMR data, and to output a final high quality dataset ready for data modeling.

Details of how to install all of the required dependencies and to set up your computing environment can be found in 'document.txt', and full documentation for the nPYc-Toolbox can be found on [read the docs](https://npyc-toolbox.readthedocs.io/en/latest/index.html)

*****TODO - Create document.txt - or maybe we can just link to the read the docs page?*****

The nPYc-Toolbox has been developed based on the quality control criteria previously described in [Dona et al. 2014](https://www.ncbi.nlm.nih.gov/pubmed/25180432)

The dataset used in this example (DEVSET U 1D NMR) is comprised of six samples of pooled human urine, aliquoted, and independently prepared and measured by 1H NMR spectroscopy. Each source sample was separately prepared and assayed thirteen times. A pooled QC sample (study reference, SR) and independent external reference (long-term reference, LTR) of a comparable matrix was also acquired to assist in assessing analytical precision. See the Metabolights Study [MTBLS694](https://www.ebi.ac.uk/metabolights/MTBLS694) for details.

# 1. Import the nPYc-Toolbox and Configure the Jupyter Notebook

*****TODO - delete next two cells once nPYc updated*****

In [1]:
toolboxPath = '/Users/cs401/Box Sync/Carolines code/phenomecentre/npyc-toolbox'

In [2]:
import sys
import pyChemometrics
sys.path.append(toolboxPath)

In [3]:
# Import the nPYc-Toolbox
import nPYc

# Import enumerations for sample type
from nPYc.enumerations import VariableType, DatasetLevel, AssayRole, SampleType

# Import normalisation objects for data normalisation
from nPYc.utilities.normalisation import NullNormaliser, TotalAreaNormaliser, ProbabilisticQuotientNormaliser

# Import matplotlib plotting, configure the Jupyter notebook to plot inline
import matplotlib.pyplot as plt
%matplotlib inline

# Set up plotly to work in offline mode with the notebook
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

# 2. Import and Preprocess NMR Data

The first step is to import the 1D NMR raw data files (Bruker format) into an nPYc-Toolbox [NMRDataset object](https://npyc-toolbox.readthedocs.io/en/latest/objects.html).

The "rawDataPath" parameter sets the location of the NMR raw data.

The NMR DevSet Dataset is located in 'DEVSET U 1D NMR raw data files'. This folder contains 94 directories, each corresponding to a spectrum acquired with the ‘noesygppr1d’ pulse sequence. The Fourier Transform, apodization and phasing have already been performed with the vendor software (i.e. TopSpin).

In [4]:
rawDataPath = './DEVSET U 1D NMR raw data files'

The “sop” parameter points to a configuration file (encoded in JSON format) which contains a set of parameters to use during data import and pre-processing, see [Configuration Files](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html) for full details.

The nPYc-Toolbox contains two default configuration files, 'GenericNMRUrine' and 'GenericNMRBlood'. These contain the recommended parameters for import and quality control of human urine and plasma/serum biofluid, respectively. For a list of all the parameters for NMR data, see Table 6 in [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/Built-in-Configuration-SOPs.html).

Since this is a urine biofluid dataset, we will use the 'GenericNMRUrine' file.

In [5]:
sop = 'GenericNMRurine'

The "pulseProgram" parameter defines the specific NMR experiment pulse program to import, in this case 'noesygppr1d' - a standard 1D experiment with a NOE water pre-saturation. 

The specific text set in 'pulseProgram' depends on the name of the pulse program (PULPROG) set when acquiring the data, and should match this exactly.

In [6]:
pulseProgram = 'noesygppr1d'

The following line creates an object representing the dataset and triggers pre-processing of the NMR spectra, including calibration to a reference peak, and interpolation of all spectra onto a common scale.

In [7]:
nmrData = nPYc.NMRDataset(rawDataPath, pulseProgram=pulseProgram, sop=sop)

In [ ]:
iplot(nPYc.plotting.plotSpectraInteractive(nmrData))

If required, users can create new configuration files, or indeed amend the existing documents with their own values, see ([Configuration Files](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html)) for more details.

However, any of the parameters present in these files ([Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/Built-in-Configuration-SOPs.html)) can also be overwritten by passing values into the data import command directly, without having to modify or generate the configuration files themselves.

For example, to interpolate the spectra to a higher resolution than the default, the argument "variableSize" can be overridden in the following manner:

```
nmrData = nPYc.NMRDataset(rawDataPath, pulseProgram=pulseProgram, sop=sop, variableSize=64000)
```

Each nPYc Dataset object contains a name that can be changed as shown in the next cell. This name will be used in the summary and visualisation reports and in the file names of the exported data.

In [ ]:
nmrData.name = 'nPYc NMR Tutorial dataset'

# 3. Import Sample Metadata and Match to Acquired Data

The default way to add sample metadata is to prepare a CSV file which follows the set of conventions as described in [Sample Metadata](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html) and match it with the acquired data using the 'addSampleInfo' method.

Although optional, this is recommended in order to make optimal use of the quality control features and visualisations provided by the nPYc-Toolbox.

An example CSV file is provided, as given in 'DEVSET U 1D NMR Basic CSV.csv:

In [ ]:
nmrData.addSampleInfo(descriptionFormat='Basic CSV', filePath='DEVSET U 1D NMR Basic CSV.csv')

As described in [Datasets](https://npyc-toolbox.readthedocs.io/en/latest/objects.html), the spectral data, sample metadata and feature metadata can be inspected directly using:

```
dataset.intensityData
dataset.sampleMetadata
dataset.featureMetadata

```

# 4. Generate Quality Control Reports

The nPYc-Toolbox offers a series of reports, pre-set visualisations comprised of text, figures and tables to describe and summarise the characteristics of the dataset, and help the user assess the overall impact of quality control decisions (ie, excluding samples or features and changing filtering criteria). 

For full details see [Reports](https://npyc-toolbox.readthedocs.io/en/latest/reports.html)

### Sample Summary Report

The first report can be used to check the expected samples against those acquired, in terms of numbers, sample type, and any samples either missing from acquisition or not recorded in the sample metadata CSV file.

In [ ]:
nPYc.reports.generateReport(nmrData, 'sample summary')

For example, for this dataset there are two samples with entries in the sample metadata CSV file, but missing from acquisition. This allows the user to quickly assess the completeness of the dataset and, for example, investigate why these samples were missing.

Based on the corresponding entries in the sample metadata CSV file, the acquired samples are categorised into different types, where 'Study Samples' comprise the main core of the study, and the others are acquired for specific roles in characterising data quality. The main QC samples here are the 'Study Reference' samples, which comprise a pool of study samples and are used to assess analytical stability accross the run. For interest we have also included some 'Long Term Reference' samples (a QC sample external to the study) and a blank, for full details see [Sample and study design nomenclature](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html#Sample-and-study-design-nomenclature).

From the 'sample summary' report, it can be seen that corresponding information is missing from the sample metadata CSV file for one sample. This sample is listed has having unknown type, and missing information. As above, this allows the user to quickly determine whether information should be added to the sample metadata CSV file for this sample, or whether the spectrum should be excluded from the final dataset (see '5. Exclude Samples and/or Features if Required' for details.

### Feature Summary Report

The feature summary report provides visualisations summarising the quality of the dataset with regards to quality control criteria previously described in [Dona et al. 2014](https://www.ncbi.nlm.nih.gov/pubmed/25180432).

In order, these consist of:
- Chemical shift calibration (Figure 1)
- Line width (Figures 2)
- Baseline consistency (Figure 3)
- Quality of solvent suppression (Figure 3)

For each parameter, acceptable default values are pre-defined in the configuration SOP, see [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/builtinSOPs.html). If different values are required, these can be set by the user as per the example in Section 2 above (2. Import and preprocess NMR data).

Any samples failing any of the above criteria are flagged in Table 1 at the end of the report. 

For full details of each of the above, see [Feature Summary Report: NMR Datasets](https://npyc-toolbox.readthedocs.io/en/latest/Reports.html#Feature-Summary-Report-NMR-Datasets)

In [ ]:
nPYc.reports.generateReport(nmrData, 'feature summary')

For example, for this dataset, there is only one sample which fails the quality control criteria on water suppression quality.

### Exporting Reports

By default all reports are output directly to the notebook (as above), however, if html copies are required these can be automatically saved to the save directory by adding the optional input argument "destinationPath".

For example, to save to the path defined in "saveDir":  

```
saveDir = '/path to save outputs'
nPYc.reports.generateReport(nmrData, 'feature summary', destinationPath=saveDir)

```

# 5. Exclude Samples and/or Features if Required

Dataset objects contain two internal 'mask' vectors, the 'sampleMask' and 'featureMask', which store whether a sample or feature respectively should be used when calculating QC metrics, visualised in the reports and finally exported, see [Sample and Feature Masks](https://npyc-toolbox.readthedocs.io/en/latest/objects.html#Sample-and-Feature-Masks).

There are several functions which modify these masks, which are useful at various stages of quality control and in preparing a final dataset for export.

### Mask Samples

The 'updateMasks' function can be used to automatically mask samples (and/or features).

For now, we do not mask any features, setting "filterFeatures=False".

By default (running the line below), samples which fail quality control (as described above) will be automatically masked, alongside any samples of unknown 'SampleType' and/or 'AssayRole':

```
nmrData.updateMasks(filterSamples=True, filterFeatures=False)
```

Here, by setting preferences with the 'sampleTypes' argument, any other samples which are not required can also be masked. In this example, we limit our dataset to study samples ('SampleType.StudySample') and study reference samples ('SampleType.StudyPool') only (excluding the long term reference, which has 'SampleType.ExternalReference' and the sample blank, which has 'SampleType.ProceduralBlank'), for full details see [Sample and study design nomenclature](https://npyc-toolbox.readthedocs.io/en/latest/samplemetadata.html#Sample-and-study-design-nomenclature) and [Enumerations](https://npyc-toolbox.readthedocs.io/en/latest/enumerations.html).

In [ ]:
nmrData.updateMasks(sampleTypes=[SampleType.StudySample, SampleType.StudyPool], filterFeatures=False)

The results of masking can be summarised using the 'sample summary' report.

In [ ]:
nPYc.reports.generateReport(nmrData, 'sample summary')

### Mask Features

For NMR datasets, typically one or more sections of the spectra known to contain unwanted or un-informative signals are removed from the data (see [Feature Filtering](https://npyc-toolbox.readthedocs.io/en/latest/Feature-Filtering.html#)).

This can be done automatically using the 'updateMasks' function.

The standard regions automatically masked are defined in the configuration SOP, see Table 6 in [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/builtinSOPs.html). For example, the default exclusion regions for urine are between -0.2 and 0.2 ppm (TSP) and between 4.7 and 4.9 ppm (water presaturation region).

As the sample mask has been specified in the previous section, here we set "filterSamples=False".

In [ ]:
nmrData.updateMasks(filterSamples=False, filterFeatures=True)

If different default regions are required to be masked, these can be amended either directly in the appropriate [Configuration SOP](https://npyc-toolbox.readthedocs.io/en/latest/configuration/configuration.html), or by the user at data import stage. 

For example, to mask a larger region around the water presaturation signal (between 4.5 and 5.0 ppm) the data could be imported with an updated "exclusionRegions" parameter:

```
nmrData = nPYc.NMRDataset(rawDataPath, pulseProgram=pulseProgram, sop=sop, exclusionRegions=[[-0.2,0.2],[4.5,5.0]])
```

Finally, additional regions can also be masked by using 'updateMasks' with the additional "exclusionRegions" parameter. For example, to also mask the region between 8.4 and 8.5 ppm the following would be run:

```
nmrData.updateMasks(filterSamples=False, filterFeatures=True, exclusionRegions=[(8.4, 8.5)])
```

The results of masking can be visualised using the 'feature summary' report.

Using "withExclusions=True" means the report is generated as if any masked features were excluded from the dataset.

In [ ]:
nPYc.reports.generateReport(nmrData, 'feature summary', withExclusions=True)

### Excluding Specific Samples and/or Features 

The 'updateMasks' function works to mask samples or features not meeting specific criteria, in addition to this, the nPYc-Toolbox also contains two additional methods to mask specific samples or features directly, 'excludeSamples' and 'excludeFeatures' respectively, see [Sample and Feature Masks](https://npyc-toolbox.readthedocs.io/en/latest/objects.html#Sample-and-Feature-Masks).

Each of these funtions takes three input arguments; firstly, a list of sample or feature identifiers; secondly, the name of the column in 'sampleMetadata' (for 'excludeSamples') or 'featureMetadata' (for 'excludeFeatures') where these identifiers can be found; and finally an optional message as to why these samples or features have been flagged for exclusion.

For example, to exclude the sample of unknown type with 'Sample File Name' 'DEVSET U 1D NMR raw data files/930':

```
nmrData.excludeSamples(['DEVSET U 1D NMR raw data files/930'], on='Sample File Name', message='Unknown type')
```

Or to exclude all features with 'ppm' > 8:

```
nmrData.excludeFeatures([nmrData.featureMetadata['ppm'][nmrData.featureMetadata['ppm'] > 8].values], on='ppm', message='ppm > 8')
```

### Permanently Exclude Masked Samples/Features

Once satisfied with the sample and feature masks, exclusions can be applied (permanently removed from the dataset) using the 'applyMasks' function.

This method should be used only when it is absolutely certain that the masked features and samples are to be removed, as the excluded data will have to be re-imported.

Before masks have been applied, they can be re-set to include all samples/features using:

```
nmrData.initialiseMasks() 
```

For details see [Sample and Feature Masks](https://npyc-toolbox.readthedocs.io/en/latest/objects.html#Sample-and-Feature-Masks)

In [ ]:
nmrData.applyMasks()

# 6. Analytical Multivariate Quality Control

The nPYc-Toolbox provides the capacity to generate a principal component analysis (PCA) model of the data (via the pyChemometrics module), and subesquently, to use this to assess data quality, identify potential sample and feature outliers, and determine any potential analytical associations with the main sources of variance in the data ([Multivariate Analysis](https://npyc-toolbox.readthedocs.io/en/latest/Multivariate-Analysis.html))

A PCA model can be generated using 'exploratoryAnalysisPCA', and there are a number of parameters which can be optimised depending on the dataset (see [PCA Model](https://npyc-toolbox.readthedocs.io/en/latest/Multivariate-Analysis.html#PCA-Model) for full details).

One key paramter is 'scaling', which divides each column in the data matrix by its respective standard deviation raised to a power of the scaling parameter. This parameter can range in value between 0 and 1, and recommended values are 0 for mean centering only, 0.5 for Pareto scaling and 1 for unit variance (UV) scaling. The outcome of PCA model will vary based on the scaling method selected, and different scaling functions can be appropriate depending on the data itself and the question being asked of the data, see [van der Berg et al. 2006](https://www.ncbi.nlm.nih.gov/pubmed/16762068)

The default scaling is unit variance ("scaling=1"), which scales every variable to have a variance of one, and thus all variables (despite their different magnitudes) become equally important in the model. For NMR, when smaller variables are more likely to be background noise, it may be that means-centering the data only ("scaling=0") can be appropriate.

Each model is cross-validated using 7-fold cross-validation and the recommended number of principal components automatically estimated based on two criteria, when either one of these is met no more components will be added and the PCA model will be returned. There criteria are:
1. 'minQ2': Q2 is the variance predicted by each component (from cross-validation), when adding a component does not improve Q2 by at least this value (default "minQ2=0.05") then no more components will be added.
2. 'maxComponents': this defines the maximum number of components (default "maxComponents=10") returned by the model (regardless of Q2 increases).

Again these parameters can be amended by adding them as input arguments to 'exploratoryAnalysisPCA'.

In [ ]:
PCAmodel = nPYc.multivariate.exploratoryAnalysisPCA(nmrData, scaling=1)

The analytical multivariate report provides visualisations summarising the largest sources of variance in the dataset (from the PCA model generated) with particular emphasis on any potential analytical sources, as defined in 'analyticalMeasurements' in the [Built-in Configuration SOPs](https://npyc-toolbox.readthedocs.io/en/latest/configuration/builtinSOPs.html). 

These consist of:
- Scree plot of variance (Figure 1)
- Scores plots coloured by sample type (Figure 2)
- Strong sample outliers (Figure 3)
- DmodX sample outliers (Figure 4)
- Loadings plots (Figure 5)
- Distribution of analytical parameters (Figure 6)
- Heatmap of potential associations between analytical parameters and the main sources of variance (Figures 7 and 8)
- Scores plots coloured by analytical parameters with potential association (Figures 9-11)

For full details of each of the above, see [Multivariate Analysis Report](https://npyc-toolbox.readthedocs.io/en/latest/Multivariate-Analysis.html#Multivariate-Analysis-Report)

In [ ]:
nPYc.reports.multivariateQCreport(nmrData, PCAmodel)

*****TODO Beatriz, please could you add some comments about what you would conclude about QC based on this report, thank you!*****

### Interactive Plots

Scores and loadings plots can also be explored interactively with the 'plotScoresInteractive' and 'plotLoadingsInteractive' functions.

**Interactive scores plot**

For example, to plot the scores plot for principal component 1 vs. pricinipal component 2 ("components=[1, 2]") with points coloured by values in nmrData.sampleMetadata['Class'] (the colour is definined by the third input argument and can be any column name in the sample metadata):

In [ ]:
data = nPYc.plotting.plotScoresInteractive(nmrData, PCAmodel, 'Class', components=[1, 2])
iplot(data)

**Interactive loadings plot**

Similarly, to plot the loadings, here for principal component 2:

In [ ]:
data = nPYc.plotting.plotLoadingsInteractive(nmrData, PCAmodel, component=2)
iplot(data)

# 7. Finalise and Export Dataset

Datasets can be exported in a variety of formats with the 'exportDataset' method (see [Exporting Data](https://npyc-toolbox.readthedocs.io/en/latest/exportingdata.html)).

By default, data will be automatically exported to the current working directory, to change this a 'saveDir' can be defined

By default datasets are exported to the current working directory, however, if files are required to be exported to a defined path, this can be done by adding the optional input argument "destinationPath".

For example, to save to the path defined in "saveDir":  

```
saveDir = '/path to save outputs'
nmrData.exportDataset(saveFormat='UnifiedCSV', destinationPath=saveDir)

```

To export a single CSV file, which contains a row for every sample, and a column for every feature, alongside all of the sample and feature specific metadata, set "saveFormat=UnifiedCSV".

In [ ]:
nmrData.exportDataset(saveFormat='UnifiedCSV')

In addition, the 'final report' provides a document summarising the contents of the dataset alongside pertinent visualisations from the 'feature summary' and 'multivariate reports':

In [ ]:
nPYc.reports.generateReport(nmrData, 'final report', pcaModel=PCAmodel, destinationPath='.')